<a href="https://colab.research.google.com/github/eunice-hong1/Eunice-Hong/blob/main/Skin_Cancer_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import cv2
from PIL import Image
import tensorflow as tf
from keras import backend as K
from keras.models import load_model
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet_v3 import models

In [ ]:
base_dir = '/content/drive/MyDrive/Image Classification/dataset/Skin cancer dataset'
train_dir = '/content/drive/MyDrive/Image Classification/dataset/Skin cancer dataset/train'
train_benign_dir = '/content/drive/MyDrive/Image Classification/dataset/Skin cancer dataset/train/benign'
train_malign_dir = '/content/drive/MyDrive/Image Classification/dataset/Skin cancer dataset/train/malignant'
test_dir = '/content/drive/MyDrive/Image Classification/dataset/Skin cancer dataset/test'
test_benign_dir = '/content/drive/MyDrive/Image Classification/dataset/Skin cancer dataset/test/benign'
test_malign_dir = '/content/drive/MyDrive/Image Classification/dataset/Skin cancer dataset/test/malignant'
valid_dir = '/content/drive/MyDrive/Image Classification/dataset/Skin cancer dataset/validation'
valid_benign_dir = '/content/drive/MyDrive/Image Classification/dataset/Skin cancer dataset/validation/benign'
valid_malign_dir = '/content/drive/MyDrive/Image Classification/dataset/Skin cancer dataset/validation/malignant'

In [ ]:
num_benign_train = len(os.listdir(train_benign_dir))
num_malignant_train = len(os.listdir(train_malign_dir))
num_benign_validaition = len(os.listdir(valid_benign_dir))
num_malignant_validation= len(os.listdir(valid_malign_dir))
num_benign_test = len(os.listdir(test_benign_dir))
num_malignant_test= len(os.listdir(test_malign_dir))

In [ ]:
print("Total Training Benign Images",num_benign_train)
print("Total Training Malignant Images",num_malignant_train)
print("--")
print("Total validation Benign Images",num_benign_validaition)
print("Total validation Malignant Images",num_malignant_validation)
print("--")
print("Total Test Benign Images", num_benign_test)
print("Total Test Malignant Images",num_malignant_test)
total_train = num_benign_train+num_malignant_train
total_validation = num_benign_validaition+num_malignant_validation
total_test = num_benign_test+num_malignant_test
print("Total Training Images",total_train)
print("--")
print("Total Validation Images",total_validation)
print("--")
print("Total Testing Images",total_test)

Total Training Benign Images 1140
Total Training Malignant Images 937
--
Total validation Benign Images 300
Total validation Malignant Images 260
--
Total Test Benign Images 360
Total Test Malignant Images 300
Total Training Images 2077
--
Total Validation Images 560
--
Total Testing Images 660


In [ ]:
IMG_SHAPE  = 224
batch_size = 32

In [ ]:
image_gen_train = ImageDataGenerator(rescale = 1./255)
train_data_gen = image_gen_train.flow_from_directory(batch_size = batch_size,
directory = train_dir,
shuffle= True,
target_size = (IMG_SHAPE,IMG_SHAPE),
class_mode = 'binary')
image_generator_validation = ImageDataGenerator(rescale=1./255)
val_data_gen = image_generator_validation.flow_from_directory(batch_size=batch_size,
directory=valid_dir,
target_size=(IMG_SHAPE, IMG_SHAPE),
class_mode='binary')
image_gen_test = ImageDataGenerator(rescale=1./255)
test_data_gen = image_gen_test.flow_from_directory(batch_size=batch_size,
directory=test_dir,
target_size=(IMG_SHAPE, IMG_SHAPE),
class_mode='binary')

Found 2077 images belonging to 2 classes.
Found 560 images belonging to 2 classes.
Found 660 images belonging to 2 classes.


In [ ]:
pre_trained_model = tf.keras.applications.VGG16(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

In [ ]:
for layer in pre_trained_model.layers:
  print(layer.name)
  layer.trainable = False  #freeze the training layers of VGG-16

input_4
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_pool


In [ ]:
last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output
x = tf.keras.layers.GlobalMaxPooling2D()(last_output)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(2, activation='sigmoid')(x)

In [ ]:
x = tf.keras.layers.Dense(3, activation='softmax')(x)

In [ ]:
model = tf.keras.Model(pre_trained_model.input,x)

In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['acc'])

In [ ]:
train_data_gen

In [ ]:
from keras.applications.mobilenet_v3 import models
vgg_classifier = model.fit(train_data_gen,
steps_per_epoch=(total_train//batch_size),
epochs = 5,validation_data=val_data_gen,
validation_steps=(total_validation//batch_size),
batch_size = batch_size,
verbose = 1)

Epoch 1/5
10/64 [===>..........................] - ETA: 18:10 - loss: 0.9143 - acc: 0.5899

In [ ]:


skin_classifier = tf.keras.Sequential([
tf.keras.layers.Conv2D(16,(3,3),activation = tf.nn.relu,input_shape=(IMG_SHAPE,IMG_SHAPE, 3)),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(32,(3,3),activation = tf.nn.relu),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(64,(3,3),activation = tf.nn.relu),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(128,(3,3),activation = tf.nn.relu),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Flatten(),
tf.keras.layers.Dropout(0.5),
tf.keras.layers.Dense(512,kernel_regularizer = tf.keras.regularizers.l2(0.001), activation = tf.nn.relu),
tf.keras.layers.Dense(2,activation = tf.nn.sigmoid)

])

In [ ]:
skin_classifier.compile(optimizer='adam', loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['acc'])

In [ ]:
skin_classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 64)       0

In [ ]:
history_skin_classifier = skin_classifier.fit(train_data_gen,
steps_per_epoch=(total_train//batch_size),
epochs = 5,
validation_data=val_data_gen,
validation_steps=(total_validation//batch_size),
batch_size = batch_size,
verbose = 1)

Epoch 1/5
64/64 [==============================] - 164s 3s/step - loss: 0.4725 - acc: 0.7770 - val_loss: 0.4073 - val_acc: 0.8015
Epoch 2/5
64/64 [==============================] - 166s 3s/step - loss: 0.4531 - acc: 0.7834 - val_loss: 0.4596 - val_acc: 0.8051
Epoch 3/5
64/64 [==============================] - 156s 2s/step - loss: 0.4732 - acc: 0.7795 - val_loss: 0.4509 - val_acc: 0.7794
Epoch 4/5
64/64 [==============================] - 163s 3s/step - loss: 0.4717 - acc: 0.7804 - val_loss: 0.3669 - val_acc: 0.8162
Epoch 5/5
64/64 [==============================] - 166s 3s/step - loss: 0.4537 - acc: 0.7912 - val_loss: 0.4122 - val_acc: 0.7941


In [ ]:
train_data_gen.class_indices

{'benign': 0, 'malignant': 1}